In [69]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import psycopg2
import scraper
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import PorterStemmer

In [2]:
# Note: if no dbname or user are specified, the default is your username.
conn = psycopg2.connect("dbname=food_db")
cur = conn.cursor()
conn.autocommit = True

In [145]:
page=1
urls = scraper.get_urls(page)

In [146]:
for url in urls:
    row = scraper.get_ingredients(url)
    query = """
    INSERT INTO
        recipes (post_date, title, foods)
        VALUES (%s, %s, %s);
    """
    cur.execute(query, row)

In [3]:
query = '''
        SELECT *
        FROM recipes
        LIMIT 30;
        '''
cur.execute(query)

In [6]:
cur.fetchone()

(27,
 datetime.datetime(2018, 2, 9, 0, 0),
 'Long Life Noodles with Shrimp and Greens',
 'teaspoon sesame oil, for drizzling stalk green onion, finely chopped cups all-purpose flour, plus more for dusting medium carrot, julienned (about 1⁄2 cup) stalk green onion, cut into 2-inch segments tablespoon plus 1 teaspoon vegetable oil, divided kosher salt cup water teaspoon minced fresh ginger tablespoon hoisin sauce teaspoon soy sauce teaspoons cornstarch pound raw shrimp, size 16/20, peeled and deveined tablespoons soy sauce cups roughly chopped greens, such as baby bok choy, yu choy, or chinese broccoli cup warm tap water large clove garlic, minced')

In [7]:
df = pd.read_sql('SELECT * FROM recipes', con=conn)

In [11]:
docs = df.foods

In [55]:
food_stops = {'pinch', 'serving', 'teaspoon', 'teaspoons', 'tablespoon', 'tablespoons', 'cup', 'cups', 
              'optional', 'taste', 'oz', 'package', 'see', 'note', 'small', 'medium', 'large', 'cut', 
              'inch', 'divided', 'pounds', 'pound', 'plus'}

In [70]:
porter = PorterStemmer()
doc4 = docs[0]
doc4 = ''.join(ch for ch in doc4 if ch in set(string.ascii_lowercase + ' '))
doc4 = word_tokenize(doc4)
sw = set(stopwords.words('english'))
sw.update(food_stops)
doc4 = [w for w in doc4 if not w in sw]
p_stems = [porter.stem(word) for word in doc4]
p_stems

['sesam',
 'oil',
 'drizzl',
 'stalk',
 'green',
 'onion',
 'fine',
 'chop',
 'allpurpos',
 'flour',
 'dust',
 'carrot',
 'julien',
 'stalk',
 'green',
 'onion',
 'segment',
 'veget',
 'oil',
 'kosher',
 'salt',
 'water',
 'minc',
 'fresh',
 'ginger',
 'hoisin',
 'sauc',
 'soy',
 'sauc',
 'cornstarch',
 'raw',
 'shrimp',
 'size',
 'peel',
 'devein',
 'soy',
 'sauc',
 'roughli',
 'chop',
 'green',
 'babi',
 'bok',
 'choy',
 'yu',
 'choy',
 'chines',
 'broccoli',
 'warm',
 'tap',
 'water',
 'clove',
 'garlic',
 'minc']

In [74]:
from process_words import clean_one_doc

In [99]:
clean_one_doc(docs[798])

['warm',
 'water',
 'saffron',
 'threads',
 'crushed',
 'powder',
 'pestle',
 'mortar',
 'unsalted',
 'butter',
 'olive',
 'oil',
 'leeks',
 'whites',
 'tender',
 'green',
 'parts',
 'halved',
 'halfmoon',
 'shapes',
 'washed',
 'well',
 'shrimp',
 'shell',
 'deveined',
 'washed',
 'sea',
 'salt',
 'herbs',
 'choice',
 'use',
 'cilantro',
 'leaves',
 'stems',
 'italian',
 'flatleaf',
 'parsley',
 'leaves',
 'mint',
 'leaves']